In [ ]:
!pip install numpy
!pip install sklearn
!pip install pandas

In [ ]:
import json
import random
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def load_data(file_path):
    file_path = 'people_combind.csv'
    data = pd.read_csv(file_path)
    return data

def distance(vecA, vecB):
    dist = (vecA - vecB) * (vecA - vecB).T
    return dist[0, 0]

def randCent(data, k):
    n = np.shape(data)[1]  # 属性的个数
    centroids = np.mat(np.zeros((k, n)))  # 初始化k个聚类中心
    for j in xrange(n):  # 初始化聚类中心每一维的坐标
        minJ = np.min(data[:, j])
        rangeJ = np.max(data[:, j]) - minJ
        # 在最大值和最小值之间随机初始化
        centroids[:, j] = minJ * np.mat(np.ones((k , 1))) + np.random.rand(k, 1) * rangeJ
    return centroids

def kmeans(data, k, centroids):
    m, n = np.shape(data) # m：样本的个数，n：特征的维度
    subCenter = np.mat(np.zeros((m, 2)))  # 初始化每一个样本所属的类别
    change = True  # 判断是否需要重新计算聚类中心
    while change == True:
        change = False  # 重置
        for i in xrange(m):
            minDist = np.inf  # 设置样本与聚类中心之间的最小的距离，初始值为争取穷
            minIndex = 0  # 所属的类别
            for j in xrange(k):
                # 计算i和每个聚类中心之间的距离
                dist = distance(data[i, ], centroids[j, ])
                if dist < minDist:
                    minDist = dist
                    minIndex = j
            # 判断是否需要改变
            if subCenter[i, 0] <> minIndex:  # 需要改变
                change = True
                subCenter[i, ] = np.mat([minIndex, minDist])
        # 重新计算聚类中心
        for j in xrange(k):
            sum_all = np.mat(np.zeros((1, n)))
            r = 0  # 每个类别中的样本的个数
            for i in xrange(m):
                if subCenter[i, 0] == j:  # 计算第j个类别
                    sum_all += data[i, ]
                    r += 1
            for z in xrange(n):
                try:
                    centroids[j, z] = sum_all[0, z] / r
                except:
                    print " r is zero"   
    return subCenter

def save_result(file_name, source):
    m, n = np.shape(source)
    f = open(file_name, "w")
    for i in xrange(m):
        tmp = []
        for j in xrange(n):
            tmp.append(str(source[i, j]))
        f.write("\t".join(tmp) + "\n")
    f.close()

In [ ]:
# class KMeans:
#     def cal_dist(self, p0, p1):
#         """
#         比較兩點的距離
#         """
#         return np.sqrt(np.sum((p0-p1)**2))
    
#     def nearest_cluster_center(self, point, cluster_centers):
#         """
#         找到距離 point 最近的中心點
#         """
#         min_dist = float("inf")
#         m = cluster_centers.shape[0]
#         for i in range(m):
#             d = self.cal_dist(point, cluster_centers[i])
#             if min_dist > d:
#                 min_dist = d
#         return min_dist 

#     def get_centroids(self, datapoints, k):
#         """
#         K-means++ 演算法，取得初始化中心點
#         """
#         clusters = np.array([random.choice(datapoints)])
#         dist = np.zeros(len(datapoints))
        
#         for i in range(k-1):
#             sum_dist = 0
#             for j, point in enumerate(datapoints):
#                 dist[j] = self.nearest_cluster_center(point, clusters)
#                 sum_dist += dist[j]
            
#             sum_dist *= random.random()
#             for j, d in enumerate(dist):
#                 sum_dist = sum_dist - d
#                 if sum_dist <= 0:
#                     clusters = np.append(clusters, [datapoints[j]], axis=0)
#                     break
        
#         return clusters
        
        
#     def kmeans_plus_plus(self, datapoints, k=2):
#         """
#         K-means 演算法
#         """
#         # 定義資料維度
#         d = datapoints.shape[1]
#         # 最大的迭代次數
#         Max_Iterations = 1000

#         cluster = np.zeros(datapoints.shape[0])
#         prev_cluster = np.ones(datapoints.shape[0])

#         cluster_centers = self.get_centroids(datapoints, k)

#         iteration = 0
#         while np.array_equal(cluster, prev_cluster) is False or iteration > Max_Iterations:
#             iteration += 1
#             prev_cluster = cluster.copy()

#             # 將每一個點做分群
#             for idx, point in enumerate(datapoints):
#                 min_dist = float("inf")
#                 for c, cluster_center in enumerate(cluster_centers):
#                     dist = self.cal_dist(point, cluster_center)
#                     if dist < min_dist:
#                         min_dist = dist  
#                         cluster[idx] = c   # 指定該點屬於哪個分群

#             # 更新分群的中心
#             for k in range(len(cluster_centers)):
#                 new_center = np.zeros(d)
#                 members = 0
#                 for point, c in zip(datapoints, cluster):
#                     if c == k:
#                         new_center += point
#                         members += 1
#                 if members > 0:
#                     new_center = new_center / members
#                 cluster_centers[k] = new_center

#         return cluster